In [10]:
# %pip install langchain
# %pip install pypdf
# %pip install rapidocr-onnxruntime
# %pip install faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 17.3 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [24]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOllama
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain.vectorstores import FAISS

In [31]:
loader = PyPDFLoader("sample_data/Orca 2.pdf", extract_images=True)
pages = loader.load_and_split()

In [32]:
embeddings = OllamaEmbeddings(model="orca2")

In [33]:
faiss_index = FAISS.from_documents(pages, embeddings)

In [34]:
retriever = faiss_index.as_retriever()

In [35]:
tool = create_retriever_tool(
    retriever,
    "search_secure_by_design",
    "Searches and returns informations regarding the state of the art of security by design.",
)
tools = [tool]

In [36]:
chat_model = ChatOllama(
    model="orca2",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

In [37]:
template = """[INST] <<SYS>> Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. <</SYS>>
{context}
Question: {question}
Helpful Answer:[/INST]"""
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

In [38]:
qa_chain = RetrievalQA.from_chain_type(
    chat_model,
    retriever=retriever,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [39]:

question = "Give me a summary of this document."
result = qa_chain({"query": question})

Hello, I am an AI assistant that can help you find information. You have asked me to summarize a document about a cautious reasoner model called Orca 2. Here is a possible summary:

Key points:
- The document describes the data and evaluation methods for a new cautious reasoner model called Orca 2, which can learn from prompt erasing and few shot data.
- The document explains how the model can generate text for various tasks that require reasoning, knowledge, language understanding, summarization, safety, bias, and comprehension.
- The document provides examples of texts generated by the model and compares them with human references and other models.

Summary:
The document introduces Orca 2, a new cautious reasoner model that can learn from different sources of data. The model can generate text for various tasks that test its abilities to reason, understand language, answer questions, summarize information, avoid errors, handle bias, and comprehend texts. The document shows some exampl

In [40]:
question = "Give me a summary of the explanation about how the model can generate text that require reasoning."
result = qa_chain({"query": question})

To summarize, the explanation provided by the user is as follows:

- The user's question is about how the model can generate text that requires reasoning.
- The user seems to be asking for a summary of the explanation given earlier in the conversation.
- The explanation involves the use of WizardLM, an instruction-tuned version of LLaMA 2, which can generate diverse and complex texts through autonomous instructions.
- Additionally, other models like Orca 1 and GPT Models are used for comparison purposes.
- Benchmarks such as AGIEval, DROP, CRASS, RACE, BBH, and GSM8K are mentioned to assess the reasoning capabilities of LLMs.
- The user also seems interested in understanding how LLMs can understand language, knowledge, and perform multi-step reasoning tasks through MMLU, ARC, and Big-Bench Hard.